Retrying Assignmnet 05 code only

## Part 1

In [8]:
import pandas as pd

In [9]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


In [10]:
df = df[df["Sex"] != "Both sexes"]
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


In [11]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [12]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','Administrative Counties','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')
#print (df.head(3))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Sex                 6464 non-null   object
 1   Single Year of Age  6464 non-null   int64 
 2   VALUE               6464 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 202.0+ KB


<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_3262/3637591706.py:5: SyntaxWarning: invalid escape sequence '\D'
  df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)


In [68]:
df_anal = pd.pivot_table(df, 'VALUE',"Single Year of Age", "Sex")
print (df_anal.head())
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

Sex                    Female       Male
Single Year of Age                      
0                   1761.6250  1850.6250
1                   1721.5625  1804.6875
2                   1810.8750  1889.7500
3                   1842.6875  1937.5625
4                   1863.6875  1980.3750


In [33]:
headers = list(df_anal.columns)
female = headers[0]
male = headers[1]
Female, Male

('Female', 'Male')

In [34]:
number_female = df_anal[female].sum()
number_female

np.float64(162786.875)

In [58]:
number_male = df_anal[male].sum()
number_male

np.float64(159034.3125)

In [35]:
df_anal

Sex,Female,Male
Single Year of Age,,
0,1761.6250,1850.6250
1,1721.5625,1804.6875
2,1810.8750,1889.7500
3,1842.6875,1937.5625
4,1863.6875,1980.3750
...,...,...
96,59.7500,20.4375
97,45.7500,13.5625
98,30.7500,8.1250


### (a) Weighted Mean by Sex

In [44]:
import numpy as np
w_mean = np.average(df_anal.index, weights=df_anal[female])
w_mean

# weighted mean for females

np.float64(38.9397958987787)

In [ ]:
w_mean = np.average(df_anal.index, weights=df_anal[male])
w_mean

# weighted mean for males

np.float64(37.7394477371039)

Females Weighted Mean = 38.9397958987787

Males Weighted Mean = 37.7394477371039

### (b) Difference Between Sexes by Age

In [70]:
df_anal["Difference"] = df_anal[female] - df_anal[male]
df_anal


Sex,Female,Male,Difference
Single Year of Age,,,
0,1761.6250,1850.6250,-89.0000
1,1721.5625,1804.6875,-83.1250
2,1810.8750,1889.7500,-78.8750
3,1842.6875,1937.5625,-94.8750
4,1863.6875,1980.3750,-116.6875
...,...,...,...
96,59.7500,20.4375,39.3125
97,45.7500,13.5625,32.1875
98,30.7500,8.1250,22.6250


## Weighted Median by Sex (Optional)

### Weighted Median Female

In [38]:
cumsum = df_anal[female].cumsum()
cumsum

Single Year of Age
0        1761.6250
1        3483.1875
2        5294.0625
3        7136.7500
4        9000.4375
          ...     
96     162652.8750
97     162698.6250
98     162729.3750
99     162750.3750
100    162786.8750
Name: Female, Length: 101, dtype: float64

In [48]:
# sum of all female
cutoff = df_anal[female].sum()/2
cutoff

df_anal[female][cumsum>=cutoff].index[0]

np.int64(39)

In [47]:
# sum of all male
cutoff = df_anal[male].sum()/2
cutoff

df_anal[male][cumsum>=cutoff].index[0]

np.int64(38)